### Extract frames

Extracting frames from a video allows you to break it down into individual images. This is useful for training a model because YOLO works with still images, not videos. It helps us analyze or process specific moments in the video and gives the model more training data to learn from.

In [3]:
# Get how many frames I can extract per second from a video file

import cv2

video_path = "video3.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Cannot open video file.")
else:
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"The video has {fps} frames per second.")

cap.release()

The video has 30.0 frames per second.


In [7]:
import cv2
import os

def extract_frames(video_path, output_folder, frame_rate=1):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Frames per second of the video
    frame_interval = max(1, fps // frame_rate)  # Interval between frames to capture
    frame_count = 0
    saved_count = 0

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Save frames at the specified frame interval
        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f"Jas_{saved_count:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            saved_count += 1

        frame_count += 1

    cap.release()
    print(f"Extraction complete. {saved_count} frames saved to {output_folder}.")

video_path = "video3.mp4"
output_folder = "Raw_frames"
extract_frames(video_path, output_folder, frame_rate=3)  # no. of frame per second


Extraction complete. 801 frames saved to Raw_frames.


In [ ]:
# Rename in case if you missed to edit the code above

import os

def rename_images(folder_path, start_index=1):
    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        return

    # Supported image file extensions
    supported_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".tiff")

    counter = start_index
    for filename in os.listdir(folder_path):
        # Check if the file has a supported image extension
        if filename.lower().endswith(supported_extensions):
            # Get the file extension
            file_extension = os.path.splitext(filename)[1]

            # Create the new file name
            new_filename = f"Ray_{counter:04d}{file_extension}"

            # Define full paths for renaming
            old_file_path = os.path.join(folder_path, filename)
            new_file_path = os.path.join(folder_path, new_filename)

            # Rename the file
            os.rename(old_file_path, new_file_path)
            print(f"Renamed: {filename} -> {new_filename}")

            counter += 1

    print("Renaming complete!")

folder_path = "Ray_outputFrames"
rename_images(folder_path, start_index=1)


Renamed: frame_0000.jpg -> Ray_0001.jpg
Renamed: frame_0001.jpg -> Ray_0002.jpg
Renamed: frame_0002.jpg -> Ray_0003.jpg
Renamed: frame_0003.jpg -> Ray_0004.jpg
Renamed: frame_0004.jpg -> Ray_0005.jpg
Renamed: frame_0005.jpg -> Ray_0006.jpg
Renamed: frame_0006.jpg -> Ray_0007.jpg
Renamed: frame_0007.jpg -> Ray_0008.jpg
Renamed: frame_0008.jpg -> Ray_0009.jpg
Renamed: frame_0009.jpg -> Ray_0010.jpg
Renamed: frame_0010.jpg -> Ray_0011.jpg
Renamed: frame_0011.jpg -> Ray_0012.jpg
Renamed: frame_0012.jpg -> Ray_0013.jpg
Renamed: frame_0013.jpg -> Ray_0014.jpg
Renamed: frame_0014.jpg -> Ray_0015.jpg
Renamed: frame_0015.jpg -> Ray_0016.jpg
Renamed: frame_0016.jpg -> Ray_0017.jpg
Renamed: frame_0017.jpg -> Ray_0018.jpg
Renamed: frame_0018.jpg -> Ray_0019.jpg
Renamed: frame_0019.jpg -> Ray_0020.jpg
Renamed: frame_0020.jpg -> Ray_0021.jpg
Renamed: frame_0021.jpg -> Ray_0022.jpg
Renamed: frame_0022.jpg -> Ray_0023.jpg
Renamed: frame_0023.jpg -> Ray_0024.jpg
Renamed: frame_0024.jpg -> Ray_0025.jpg


### Compresss 

Compressing images reduces their file size while keeping enough quality for your model to learn from. It saves storage space, speeds up data loading, and reduces training time without affecting the accuracy too much.

In [ ]:
from PIL import Image
import os

def compress_image(input_image_path, output_image_path, quality=85):
    try:
        # Open the image file
        with Image.open(input_image_path) as img:
            # Save the image with the specified quality
            img.save(output_image_path, "JPEG", quality=quality)
        print(f"Image saved successfully to {output_image_path} with {quality}% quality.")
    except Exception as e:
        print(f"Error: {e}")

input_folder = "Frames/Ray"
output_folder = "Compressed/All"

os.makedirs(output_folder, exist_ok=True)

# Loop through all images in the input folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        input_image_path = os.path.join(input_folder, filename)
        output_image_path = os.path.join(output_folder, filename)

        # Compress the image
        compress_image(input_image_path, output_image_path, quality=85)


Image saved successfully to Compressed/All\Ray_0001.jpg with 85% quality.
Image saved successfully to Compressed/All\Ray_0002.jpg with 85% quality.
Image saved successfully to Compressed/All\Ray_0003.jpg with 85% quality.
Image saved successfully to Compressed/All\Ray_0004.jpg with 85% quality.
Image saved successfully to Compressed/All\Ray_0005.jpg with 85% quality.
Image saved successfully to Compressed/All\Ray_0006.jpg with 85% quality.
Image saved successfully to Compressed/All\Ray_0007.jpg with 85% quality.
Image saved successfully to Compressed/All\Ray_0008.jpg with 85% quality.
Image saved successfully to Compressed/All\Ray_0009.jpg with 85% quality.
Image saved successfully to Compressed/All\Ray_0010.jpg with 85% quality.
Image saved successfully to Compressed/All\Ray_0011.jpg with 85% quality.
Image saved successfully to Compressed/All\Ray_0012.jpg with 85% quality.
Image saved successfully to Compressed/All\Ray_0013.jpg with 85% quality.
Image saved successfully to Compressed

### Normalize the images

Normalization is essential for model training because it helps the model learn better and faster by making the data consistent and reducing bias from large or uneven pixel values.

In [29]:
import cv2
import os
import numpy as np

def normalize_images(input_folder, output_folder, normalization_range=(0, 1)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    min_val, max_val = normalization_range
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            img_path = os.path.join(input_folder, filename)
            img = cv2.imread(img_path)

            if img is None:
                print(f"Failed to load {filename}. Skipping...")
                continue

            # Normalize the image
            img_normalized = cv2.normalize(img.astype('float32'), None, alpha=min_val, beta=max_val, norm_type=cv2.NORM_MINMAX)

            # Scale back to 8-bit values for saving
            img_normalized = (img_normalized * 255).astype('uint8') if normalization_range == (0, 1) else img_normalized.astype('uint8')

            # Save normalized image
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, img_normalized)

    print(f"Normalization complete. Normalized images saved to {output_folder}.")

# Example usage:
input_folder = "Compressed/All"
output_folder = "Normalized/All"
normalize_images(input_folder, output_folder, normalization_range=(0, 1))  # Normalize to range [0, 1]


Normalization complete. Normalized images saved to Normalized/All.
